In [1]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import time 
from datetime import datetime
import os
from os import path
from glob import glob

import ijson
from tqdm import tqdm

# Import JSON

In [126]:
zip_dir  = "../data/zip"

files = glob(os.path.join(zip_dir, '*'))
for filename in files:
    if '.zip' in filename:
        pass
    elif '.json' in filename:
        pass
    else:
        newfile = filename+'.json'
        os.rename(filename, newfile)

jsons = glob(os.path.join(zip_dir, '*.json'))

In [129]:
# jsons = ['../data/part_v003_o001_r_00000.json', ..., '../data/part_v003_o001_r_00001.json']

dfrts = []
dfors = []
count = 1
for i, json in enumerate(jsons):
    # Instance Preparation
    dates = []
    tweets = []
    user_ids = []
    rts = []
    
    start = time.time()
    # Load jsoten file: date and tweet
    with open(json, 'r', encoding='utf8') as file:
        pet_parse = ijson.parse(file, multiple_values=True)
        for prefix, event, value  in pet_parse:
            # Date
            if prefix == 'created_at':
                dates.append(datetime.strptime(value, '%a %b %d %H:%M:%S +0000 %Y'))
            # Tweet 
            if prefix == 'text':
                tweets.append(value.replace('\n', '').replace(' ', '').replace('\t', '').replace('　', '')) # Delte space and indet
            # User id 
            if prefix == 'user.id':
                user_ids.append(value)
            # RT Flag
            if len(dates)-1 == len(rts) and prefix == 'retweeted_status':
                rts.append(True)
            if len(dates)-2 == len(rts):
                rts.append(False)
                
        if len(dates) != len(rts):
                rts.append(False)
#
    print ("Loading json elapsed_time:{0}".format(time.time() - start) + "[sec]")
    
#     print(len(dates), len(tweets), len(user_ids), len(rts))
    
    data = np.vstack([user_ids, dates, tweets, rts]).T
    df = pd.DataFrame(data, columns=['user_id', 'date', 'tweet', 'RT_flag'])
    
    # Date
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hour'] = df['date'].dt.hour
    df['time'] = df['date'].dt.time
    df['date'] = df['date'].dt.date
    
    # divide df by Original tweet and RT 
    dfrt = df[df['RT_flag'] == True]
    dfor = df[df['RT_flag'] == False]
    
#     # Save to csv file
#     dfor.to_csv('../data/new_dfs2/dfor'+str(i)+'.csv')
#     dfrt.to_csv('../data/new_dfs2/dfrt'+str(i)+'.csv')


Loading json elapsed_time:494.77058386802673[sec]


# READ CSv

In [3]:
zip_dir  = "../data/new_dfs"
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfors = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
#     df = df.dropna(subset=['tweet'])
    dfors.append(df)
    

# Check day data

In [82]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_day(dfors, filetype, savedir, key_lang):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')
    
    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'
    
    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
            
    # Find tweets
    for i, dfor in tqdm(enumerate(dfors)):
         # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        # Add columns of keywords whose cell have 1 if this tweet includes a keyword 
        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        d4 = dfor_c[dfor_c.V4 == 1]
        
        # Finally Connect dfs
        if i == 0:
            dfnew = d4
        else:
            dfnew = pd.concat([dfnew, d4], axis=0)
    
    
    # DF Only V4
    d4 = dfnew[dfnew.V4 == 1].drop(['V1', 'V2', 'V3', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20'], axis = 1)
    # Remove 'エロい'
    d4_without_eroi = d4[~d4['tweet'].str.contains('エロい', na = False)]

     # Make rows
    uni_dates = d4_without_eroi['date'].tolist()
    uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
    rows = []
    for date in uni_dates:
        r = d4_without_eroi.groupby('date').get_group(date).V4.sum()
        rows.append(r)
    data = np.array([uni_dates, rows]).T
    # columns
    cols = ['date', 'V4']
    # Create DataFrame
    dft = pd.DataFrame(data, columns=cols).groupby('date').sum()
    
    # Save to csv
    outname = 'V4b.csv'
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)
    savename = os.path.join(outdir, outname)
    dft.to_csv(savename)
            
    return dft

# Run day

In [83]:
# OR
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
filetypes = ['V']
for filetype in filetypes:
    d4_without_eroi = findtwt_day(dfors, filetype=filetype, savedir='../new_results2/orjp/day', key_lang='jp')

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10it [00:44,  4.45s/it]


In [56]:
len(d4_without_eroi)

89038

In [57]:
d4_without_eroi.groupby('date').get_group('2020-03-22').V4.sum()

2840

In [80]:
 # Make rows
uni_dates = d4_without_eroi['date'].tolist()
uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
rows = []
for date in uni_dates:
    r = d4_without_eroi.groupby('date').get_group(date).V4.sum()
    rows.append(r)
data = np.array([uni_dates, rows]).T
# columns
cols = ['date', 'V4']
# Create DataFrame
dft = pd.DataFrame(data, columns=cols).groupby('date').sum()

In [74]:
cols = ['date', 'V4']
dft = pd.DataFrame(data, columns=cols)
dft.groupby('date').sum()

In [81]:
dft

,V4
date,
2020-02-29,609
2020-03-01,3065
2020-03-02,2909
2020-03-03,2862
2020-03-04,2699
2020-03-05,2726
2020-03-06,2656
2020-03-07,2648
2020-03-08,3142


In [48]:
dfnew.head()

,user_id,date,tweet,RT_flag,year,month,day,hour,time,V1,...,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20
270,1107552161649483776,2020-03-31,3月、確定申告して、確定申告して、確定申告して、確定申告して、BLEACHの大プロジェクト発...,False,2020,3,31,14,14:59:25,0,...,0,0,0,0,0,0,0,0,0,0
382,201245935,2020-03-31,なんと…まだ可能性を残してくれているんだ…!!希望は捨てずに胸の中に大事にしまっておこう✨す...,False,2020,3,31,14,14:59:08,0,...,0,0,0,0,0,0,0,0,0,0
429,1007141454102159360,2020-03-31,カメラを向けられると目線を逸らしたり、手が動いてしまったり。沢山感じていることが多い彼だから...,False,2020,3,31,14,14:59:02,0,...,0,0,0,0,0,0,0,0,0,0
482,716926266713051137,2020-03-31,お祝いで活力が増した気がします。ほんまに今日一日いろいろありがとうございました😊わいは明日か...,False,2020,3,31,14,14:58:55,0,...,0,0,0,0,0,0,0,0,0,0
488,961735156577878016,2020-03-31,去年の一年で得た学び→「『〇〇公式供給のおかげで現実生きれる😂』的言説は多いが、いかにオタク...,False,2020,3,31,14,14:58:54,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
d4 = dfnew[dfnew.V4 == 1].drop(['V1', 'V2', 'V3', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20'], axis = 1 )

In [76]:
d4.date.value_counts()

2020-03-15    11510
2020-03-29     3449
2020-03-09     3246
2020-03-30     3238
2020-03-08     3144
2020-03-12     3103
2020-03-11     3102
2020-03-01     3067
2020-03-27     3001
2020-03-17     2919
2020-03-02     2909
2020-03-10     2904
2020-03-26     2893
2020-03-03     2864
2020-03-18     2853
2020-03-23     2846
2020-03-22     2840
2020-03-24     2802
2020-03-25     2758
2020-03-05     2726
2020-03-16     2724
2020-03-13     2723
2020-03-19     2711
2020-03-04     2700
2020-03-14     2696
2020-03-28     2674
2020-03-06     2657
2020-03-07     2649
2020-03-20     2413
2020-03-21     2406
2020-03-31     2393
2020-02-29      609
Name: date, dtype: int64

In [77]:
# 3-15
d315 = d4[d4.date == '2020-02-29']
print('total tweets:', len(d315))
twtset = set(d315.tweet)
print('Unique tweets:', len(twtset))
print('Unique twt rate', len(twtset)/len(d315))

total tweets: 609
Unique tweets: 608
Unique twt rate 0.9983579638752053


In [78]:
# # 3-16(other days)
# d316= d4[d4.date == '2020-03-16']
# print('total tweets:', len(d316))
# twtset = set(d316.tweet)
# print('Unique tweets:', len(twtset))
# print('Unique twt  rate', len(twtset)/len(d316))

total tweets: 2724
Unique tweets: 2596
Unique twt  rate 0.9530102790014684


In [37]:
# Remove 'エロい'
d315_wo_eroi = d315[~d315['tweet'].str.contains('エロい', na = False)]

In [41]:
# 3-15 new 
print('total tweets:', len(d315))
twtset_new = set(d315_wo_eroi.tweet)
print('Unique tweets:', len(twtset_new))
print('Unique twt rate', len(twtset_new)/len(d315_new))

total tweets: 11510
Unique tweets: 3010
Unique twt rate 0.9885057471264368


# V4の処理

# Check

In [20]:
# Read hour csv file
zip_dir  = "../new_results/rtjp/hour"
csvfiles = glob(os.path.join(zip_dir, 'T*'))

thours = []
for csvfile in csvfiles:
    t = pd.read_csv(csvfile)
#     df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
#     df = df.dropna(subset=['tweet'])
    thours.append(t)
    
# date list
dates = [csvfile.split('/')[-1].split('.')[0][1:] for csvfile in csvfiles] # ['2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04']

# day csv
tday = pd.read_csv("../new_results/rtjp/day/T_original.csv").set_index('date')

# cols
cols = list(tday.columns) # ['T1', 'T2', 'T3', 'T4', 'T5']

# 一致をチェック
for t, date in zip(thours, dates):
    for col in cols:
        if t[col].sum() == tday.loc[date, col]:
            pass
        else:
            print(date, col, t[col].sum() - tday.loc[date, col])

In [ ]:
zip_dir  = "../data/new_dfs2"
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfors = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
#     df = df.dropna(subset=['tweet'])
    dfors.append(df)
    

In [3]:
A1 = pd.read_csv('../new_results/orjp/day/A_original.csv')
A2 = pd.read_csv('../new_results_jst/orjp/day/A_original.csv')

In [9]:
A1.A4.sum() == A2.A4.sum()


True